## Webscraping dos discursos de Bolsonaro

Neste notebook faço o scraping do código fonte da primeira página com os discursos, realizo a limpeza para coletar os links em que estão os discursos e extraio o texto contidos nas páginas destes links (ou seja, a transcrição dos discursos em si). 

Os próximos passos serão:

1) acabar a limpeza dos textos dos discursos, que ainda têm alguns códigos html no meio. Preciso criar uma regular expression bem completa para eliminar todo o código restante.

2) criar um crawler para paginação para fazer o scraping e coleta de texto das demais páginas. Comecei a tentar uma coleta direta com um crawler usando selenium e beautiful soup mas estou encontrando um problema ao tentar tratar os bs4 objects salvos dentro de uma lista. Esta coleta está em um notebook separado (webscraping_bolsonro_paginacao neste repositório).

### Fonte dos dados:
Site do Planalto: https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos

Última coleta realizada em 19 de agosto de 2020

#### Carregando os pacotes

In [1]:
# Manipulação de dados

import os
import sys
import re
import time
import string
import requests
import datetime
import numpy as np
import pandas as pd

#Web scraping e processamento

from collections import Counter
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
#Criando o driver e coletando o código fonte da primeira página.
#Estou usando o driver do Safari, meu browser padrão
driver=webdriver.Safari()
driver.get("https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos")


In [3]:
#Fazendo o parse do html com beautiful soup
fonte=BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
#Verificando o tipo do objeto
type(fonte)

bs4.BeautifulSoup

In [5]:
#Abri a página no chrome e usei o inspect para verificar qual era a 
#classe html em que estava os links que levam para as páginas com as 
#transcrições. Aqui uso o beautiful soup para selecionar a classe de 
#interesse e salvo os links em uma lista

links=[]
for link in fonte.find_all('a', class_="summary url"):
    links.append(link.get('href'))
print(links)

['https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-durante-a-inauguracao-da-estacao-radar-de-corumba-corumba-ms', 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-durante-a-inauguracao-da-usina-termoeletrica-porto-de-sergipe-i-aracaju-se', 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-durante-cerimonia-de-brevetacao-dos-novos-paraquedistas-rio-de-janeiro-rj', 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-durante-solenidade-de-passagem-do-comando-militar-do-leste-rio-de-janeiro-rj', 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2020/discurso-do-presidente-da-republica-jair-bolsonaro-durante-a-inauguracao-da-escola-civico-militar-rio-de-janeiro-rj', 'https://www.gov.br/planalt

In [6]:
# Definindo a função para fazer o scraping do texto do discurso do 
#código fonte

def extrai_discurso(urls):
    driver = webdriver.Safari()
    doc_source = []
    for i in urls:
        driver.get(i)
        time.sleep(5)
        doc_source.append(BeautifulSoup(driver.page_source, 'html.parser'))
    driver.close()
    return doc_source

In [7]:
# Aplicando a função à lista de links
discursos = extrai_discurso(links)

In [8]:
# Função para extrair o texto dos discursos dos links. Uso o beautiful
#soup para encontrar as tags html de parágrafo ('p') e salvo os textos
#em lista
def extrai_texto(source_object):
    text_list = []
    for s in source_object:
        text = s.find_all('p')
        text_list.append(text)
    return text_list

In [9]:
# Loop para extração do texto. Salvo os textos em lista.
discurso_lista = []
for i in extrai_texto(discursos):
    discurso_lista.append(len(i))

In [10]:
# Convertendo para array numpy
discurso_array = np.array(discurso_lista)

In [11]:
# Extrai os discursos
primeiros_discursos = extrai_texto(discursos)

In [12]:
#Verificando quantos discursos foram coletados
len(primeiros_discursos)

30

In [13]:
#Dando uma olhada nos textos
primeiros_discursos

[[<p dir="ltr" style="text-align: justify; ">Corumbá/MS, 18 de agosto de 2020</p>,
  <p style="text-align: justify; "><span><span> </span></span></p>,
  <p dir="ltr" style="text-align: justify; "><span>Obrigado! </span></p>,
  <p dir="ltr" style="text-align: justify; "><span>Primeiro eu quero justificar o que é o capacete de guavira. Quando eu servi em Nioaque, tinha uns soldados que cometiam algumas transgressões disciplinares, então da mais grave a mais leve, e eu criei uma super leve. Quando a punição não se enquadrava no regulamento disciplinar no Exército, ela passava a ser um capacete de guavira, longe de ser uma punição para o governador, que é meu ex colega de Câmara dos Deputados, o qual eu tenho gratas recordações.</span></p>,
  <p dir="ltr" style="text-align: justify; "><span>Meus senhores, minhas senhoras,</span></p>,
  <p dir="ltr" style="text-align: justify; "><span>Prezado governador, prezado prefeito Marcelo Iunes, aqui de nossa querida Corumbá, Senhor ministro Fernando

### Limpeza do texto

In [47]:
#Os textos estão entre as tags <span></span>. Aqui estou tentando 
#limpar para pegar apenas o texto entre essas tags. No entanto,
#o texto está vindo duplicado. Provavelmente preciso aplicar o group
#ao find.all, porém estou tendo porblema pois o pyhton diz que group
#não pode ser aplicado à listas.
strings=[]
for d in primeiros_discursos:
    matches=re.findall(r'<span>.+?</span>', str(d))
    strings.append(matches)
    print(strings)

    

[['<span><span>\xa0</span>', '<span>Obrigado!\xa0</span>', '<span>Primeiro eu quero justificar o que é o capacete de guavira. Quando eu servi em Nioaque, tinha uns soldados que cometiam algumas transgressões disciplinares, então da mais grave a mais leve, e eu criei uma super leve. Quando a punição não se enquadrava no regulamento disciplinar no Exército, ela passava a ser um capacete de guavira, longe de ser uma punição para o governador, que é meu ex colega de Câmara dos Deputados, o qual eu tenho gratas recordações.</span>', '<span>Meus senhores, minhas senhoras,</span>', '<span>Prezado governador, prezado prefeito Marcelo Iunes, aqui de nossa querida Corumbá, Senhor ministro Fernando Azevedo, da Defesa, Senhor ministro Augusto Heleno, do GSI.</span>', '<span>Deixei por último, por ser a mais importante, nossa querida ministra Tereza Cristina, o orgulho do agronegócio, um orgulho não apenas para o Mato Grosso do Sul, bem como para o nosso grande Brasil. Parabéns pelo trabalho maravi

In [41]:
len(strings)

30

In [48]:
#Vou tentar rodar os mesmos códigos em apenas um discurso em um objeto 
#do tipo string para ver se funciona
primeiros=str(primeiros_discursos[:1])

In [55]:
l_primeiro=re.findall(r'<span>.+?</span>', primeiros)
print(l_primeiros)

#O código funciona com apenas um texto. O problema está ocorrendo
#no momento da iteração na lista. O próximo passo será remover as tags 
#que ainda estão nesse texto e ver como aplicar a limpeza a toda a
#lista de discursos.

['<span><span>\xa0</span>', '<span>Obrigado!\xa0</span>', '<span>Primeiro eu quero justificar o que é o capacete de guavira. Quando eu servi em Nioaque, tinha uns soldados que cometiam algumas transgressões disciplinares, então da mais grave a mais leve, e eu criei uma super leve. Quando a punição não se enquadrava no regulamento disciplinar no Exército, ela passava a ser um capacete de guavira, longe de ser uma punição para o governador, que é meu ex colega de Câmara dos Deputados, o qual eu tenho gratas recordações.</span>', '<span>Meus senhores, minhas senhoras,</span>', '<span>Prezado governador, prezado prefeito Marcelo Iunes, aqui de nossa querida Corumbá, Senhor ministro Fernando Azevedo, da Defesa, Senhor ministro Augusto Heleno, do GSI.</span>', '<span>Deixei por último, por ser a mais importante, nossa querida ministra Tereza Cristina, o orgulho do agronegócio, um orgulho não apenas para o Mato Grosso do Sul, bem como para o nosso grande Brasil. Parabéns pelo trabalho maravil